<a href="https://colab.research.google.com/github/HedersonSantos/SmartHome/blob/main/automl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/code/cbeaud/smart-home-s-temperature-evalml/notebook?scriptVersionId=95672098

In [1]:
!pip install fast_ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install evalml==0.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline
import seaborn as sns
from datetime import datetime
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None)

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error
from fast_ml.feature_engineering import FeatureEngineering_DateTime

from evalml.automl import AutoMLSearch

In [4]:
!wget https://raw.githubusercontent.com/HedersonSantos/SmartHome/main/train.csv
!wget https://raw.githubusercontent.com/HedersonSantos/SmartHome/main/test.csv
!wget https://raw.githubusercontent.com/HedersonSantos/SmartHome/main/sample_submission.csv

--2022-06-02 18:38:32--  https://raw.githubusercontent.com/HedersonSantos/SmartHome/main/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 360467 (352K) [text/plain]
Saving to: ‘train.csv.2’

train.csv.2         100%[===================>] 352.02K  --.-KB/s    in 0.02s   

2022-06-02 18:38:32 (14.4 MB/s) - ‘train.csv.2’ saved [360467/360467]

--2022-06-02 18:38:32--  https://raw.githubusercontent.com/HedersonSantos/SmartHome/main/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170613 (167K) [text/plain]
Saving to: 

In [5]:
def plotaSeries(treino, teste, predito):
  if type(treino) == pd.core.series.Series:
    treino.plot(legend=True,label='Treino')
  if type(teste) == pd.core.series.Series:
    teste.plot(legend=True,label='Teste',figsize=(16,8))
  if  type(predito)==pd.core.series.Series:
    predito.plot(legend=True) 
  else:
    for p in predito.columns:
      predito[p].plot(legend=True)

In [6]:
df_treino = pd.read_csv('train.csv')
df_teste = pd.read_csv('test.csv')


In [7]:
dt_fe = FeatureEngineering_DateTime()

def features_eng(data):
    # Step 1
    data['Date']=data['Date'].copy() + ' ' + data['Time']
    data['Date']=pd.to_datetime(data['Date'])
    data=data.drop(['Time'],axis=1)
    
    # Step 2
    dt_fe.fit(data, datetime_variables=['Date'])
    data = dt_fe.transform(data)
    
    return data

In [8]:
train = features_eng(df_treino)
test = features_eng(df_teste)

In [9]:
nunique_train=train.nunique().reset_index()
remove_col=nunique_train[(nunique_train[0]==len(train)) | (nunique_train[0]==0) | (nunique_train[0]==1) ]['index'].tolist()
remove_col

['Id',
 'Date',
 'Date:year',
 'Date:is_month_start',
 'Date:is_quarter_start',
 'Date:is_year_end',
 'Date:is_year_start',
 'Date:second']

In [10]:
def data_clean(data):
    # Step 1
    data=data.drop(remove_col,axis=1)
    
    # Step 2 (Data preprocessing)
    data['Date:is_weekend']=data['Date:is_weekend'].replace({False: 0, True: 1})
    data['Date:is_quarter_end']=data['Date:is_quarter_end'].replace({False: 0, True: 1})
    data['Date:is_month_end']=data['Date:is_month_end'].replace({False: 0, True: 1})
    
    # Step 3
    data=data.drop(['Date:time'],axis=1)
    
    # Step 4
    data['Date:day_part'] = data['Date:day_part'].fillna(value=np.nan)
    
    # Step 5
    data['Date:day_part'] = data['Date:day_part'].fillna(value='midnight')
    
    # Step 6
    data['Date:day_part'] = data['Date:day_part'].replace({'midnight':0, 'dawn':1, 'early morning':2, 
                                                               'late morning':3, 'noon':4, 'afternoon':5,
                                                               'evening':6, 'night':7})
    
    return data

In [11]:
train = data_clean(train)
test = data_clean(test)

In [12]:
import re

train = train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
test = test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

train.head()

,CO2_dinningroom,CO2_room,Relative_humidity_dinningroom,Relative_humidity_room,Lighting_dinningroom,Lighting_room,Meteo_Rain,Meteo_Sun_dusk,Meteo_Wind,Meteo_Sun_light_in_west_facade,Meteo_Sun_light_in_east_facade,Meteo_Sun_light_in_south_facade,Meteo_Sun_irradiance,Outdoor_relative_humidity_Sensor,Day_of_the_week,Indoor_temperature_room,Datequarter,Datemonth,Dateday,Dateday_of_week,Dateday_of_year,Dateweekofyear,Dateis_month_end,Dateis_quarter_end,Datehour,Dateminute,Dateis_weekend,Dateday_part
0,216.560,221.920,39.9125,42.4150,81.6650,113.520,0.0,623.360,1.42625,9690.24,12604.20,95436.8,758.880,48.3750,2.0,17.8275,1,3,13,1,73,11,0,0,11,45,0,4
1,219.947,220.363,39.9267,42.2453,81.7413,113.605,0.0,623.211,1.59200,11022.00,10787.20,95436.8,762.069,47.8080,2.0,18.1207,1,3,13,1,73,11,0,0,12,0,0,4
2,219.403,218.933,39.7720,42.2267,81.4240,113.600,0.0,622.656,1.89133,13960.50,9669.63,95398.6,766.251,47.4320,2.0,18.4367,1,3,13,1,73,11,0,0,12,15,0,4
3,218.613,217.045,39.7760,42.0987,81.5013,113.344,0.0,622.571,1.82800,18511.20,9648.13,95360.3,766.037,47.0240,2.0,18.7513,1,3,13,1,73,11,0,0,12,30,0,4
4,217.714,216.080,39.7757,42.0686,81.4657,113.034,0.0,622.400,2.36071,26349.00,9208.32,95354.9,762.743,45.4743,2.0,19.0414,1,3,13,1,73,11,0,0,12,45,0,4


In [13]:
X = train.copy()
y = X.pop('Indoor_temperature_room')
from sklearn.model_selection import train_test_split
X_, X_test, y_, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

In [14]:
# run model
model_evalml = AutoMLSearch(X_train=X_, y_train=y_, problem_type='regression', max_time=1200) # 20 minutes
model_evalml.search()

Generating pipelines to search over...
8 pipelines ready for search.

*****************************
* Beginning pipeline search *
*****************************

Optimizing for R2. 
Greater score is better.

Using SequentialEngine to train and score pipelines.
Will stop searching for new pipelines after 1200 seconds.

Allowed model families: decision_tree, random_forest, linear_model, lightgbm, xgboost, catboost, extra_trees



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Evaluating Baseline Pipeline: Mean Baseline Regression Pipeline
Mean Baseline Regression Pipeline:
	Starting cross validation
	Finished cross validation - mean R2: -0.001
	High coefficient of variation (cv >= 0.2) within cross validation scores.
	Mean Baseline Regression Pipeline may not perform as estimated on unseen data.

*****************************
* Evaluating Batch Number 1 *
*****************************

Linear Regressor w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean R2: 0.756
Decision Tree Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.867
Random Forest Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.929
LightGBM Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.934


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.419e+03, tolerance: 1.153e+00

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.410e+03, tolerance: 1.189e+00

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.409e+03, tolerance: 1.204e+00



Elastic Net Regressor w/ Imputer + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean R2: 0.746
XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.984
Extra Trees Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.911
CatBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.329

*****************************
* Evaluating Batch Number 2 *
*****************************

XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.947
XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.980
XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.950
XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross validation - mean R2: 0.947
XGBoost Regressor w/ Imputer:
	Starting cross validation
	Finished cross 

In [15]:
model_evalml.rankings

,id,pipeline_name,search_order,mean_cv_score,standard_deviation_cv_score,validation_score,percent_better_than_baseline,high_variance_cv,parameters
0,51,XGBoost Regressor w/ Imputer,51,0.988473,0.001576,0.986837,153638.713164,False,"{'Imputer': {'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'XGBoost Regressor': {'eta': 0.03842638804730825, 'max_depth': 12, 'min_child_weight': 9.895364542533038, 'n_estimators': 965, 'n_jobs': -1}}"
2,55,LightGBM Regressor w/ Imputer,55,0.987798,0.002653,0.985423,153533.834718,False,"{'Imputer': {'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'median', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'LightGBM Regressor': {'boosting_type': 'dart', 'learning_rate': 0.41436917385778055, 'n_estimators': 68, 'max_depth': 7, 'num_leaves': 81, 'min_child_samples': 14, 'n_jobs': -1, 'bagging_freq': 0, 'bagging_fraction': 0.6747526475066985}}"
10,45,CatBoost Regressor w/ Imputer,45,0.985020,0.004844,0.986504,153102.255586,False,"{'Imputer': {'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'most_frequent', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'CatBoost Regressor': {'n_estimators': 91, 'eta': 0.47997769239788496, 'max_depth': 5, 'bootstrap_type': None, 'silent': False, 'allow_writing_files': False, 'n_jobs': -1}}"
24,63,Random Forest Regressor w/ Imputer,63,0.982622,0.002769,0.980187,152729.895728,False,"{'Imputer': {'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'most_frequent', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'Random Forest Regressor': {'n_estimators': 109, 'max_depth': 25, 'n_jobs': -1}}"
85,108,Extra Trees Regressor w/ Imputer,108,0.946215,0.003893,0.943454,147074.837095,False,"{'Imputer': {'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'most_frequent', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'Extra Trees Regressor': {'n_estimators': 575, 'max_features': 'auto', 'max_depth': 7, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_jobs': -1}}"
91,150,Decision Tree Regressor w/ Imputer,150,0.941003,0.007745,0.948497,146265.158489,False,"{'Imputer': {'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'Decision Tree Regressor': {'criterion': 'mse', 'max_features': 'sqrt', 'max_depth': 10, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0}}"
148,195,Linear Regressor w/ Imputer + Standard Scaler,195,0.755941,0.013095,0.770700,117519.670385,False,"{'Imputer': {'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'Linear Regressor': {'fit_intercept': True, 'normalize': True, 'n_jobs': -1}}"
170,5,Elastic Net Regressor w/ Imputer + Standard Scaler,5,0.746073,0.013069,0.760777,115986.831536,False,"{'Imputer': {'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'Elastic Net Regressor': {'alpha': 0.0001, 'l1_ratio': 0.15, 'max_iter': 1000, 'normalize': False}}"
212,0,Mean Baseline Regression Pipeline,0,-0.000644,0.000567,-0.000684,0.000000,True,{'Baseline Regressor': {'strategy': 'mean'}}


In [16]:
import math
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def evaluates_model(y_test, y_pred):
    print("*"*12, "Evaluations", "*"*12, '\n')
    
    print("MAE model :", mean_absolute_error(y_test, y_pred))
    
    print("MSE model :", mean_squared_error(y_test, y_pred))
    
    print("R2_Score model :", r2_score(y_test, y_pred))
    
    mse_1 = np.square(np.subtract(y_test,y_pred)).mean() 
    
    print("RMSE model :", math.sqrt(mse_1))

In [17]:
model_evalml.best_pipeline

pipeline = RegressionPipeline(component_graph={'Imputer': ['Imputer'], 'XGBoost Regressor': ['XGBoost Regressor', 'Imputer.x']}, parameters={'Imputer':{'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'categorical_fill_value': None, 'numeric_fill_value': None}, 'XGBoost Regressor':{'eta': 0.03842638804730825, 'max_depth': 12, 'min_child_weight': 9.895364542533038, 'n_estimators': 965, 'n_jobs': -1}}, random_seed=0)

In [18]:
pred = model_evalml.best_pipeline.predict(X_test)
evaluates_model(y_test, pred)

************ Evaluations ************ 

MAE model : 0.17218864650622848
MSE model : 0.08255147850554921
R2_Score model : 0.989273557746614
RMSE model : 4.044275249392217


In [19]:
y_predict = model_evalml.best_pipeline.predict(test)

len(y_predict)

1373